In [3]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from selenium.webdriver.chrome.options import Options 
from selenium.webdriver.common.keys import Keys
import chromedriver_binary
import gspread
import time
import cv2
import numpy as np
from PIL import Image
import requests

from oauth2client.service_account import ServiceAccountCredentials 
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
credentials = ServiceAccountCredentials.from_json_keyfile_name('suraping-381608-4562bacb42ee.json', scope)

def download_img(image, filename):
    r = requests.get(url=image, cookies=cookie, headers=headers, stream=True)
    if r.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(r.content)

date = 0 #当日は0、前日は1・・・
date2 = "08/19"
pmc = "08022017" # 店舗ID
SPREADSHEET_KEY = '1UI1bfGF8plOnDcJaIAPO7K__E22AeYNhHRAtIMrZzuo' # スプレッドシートのID
pstart = 1 # パチンコの1台目
sstart = 281 # スロットの1台目
wait = 0.01

user_id = "tomosvtg.1988+789@gmail.com "
user_pw = "paruparu23"

Template_p_path = "site777_2/template_p.png"
Template_s_path = "site777_2/template_s.png"
dot_point_p = 130000 / 432
dot_point_px = 60000 / 234
dot_point_s = 20000 / 432

mobile_emulation = { "deviceName": "iPhone 12 Pro" }

headers = {
    "User-Agent": "Mozilla/5.0 (iPhone; CPU iPhone OS 16_1 like Mac OS X) AppleWebKit/605.1.15 (KHTML, like Gecko) CriOS/108.0.5359.112 Mobile/15E148 Safari/604.1",
    "Referer" : "https://m.site777.jp/f/D3000.do?pmc=22021007&gc=2&dn=5019&urt=2173&mdc=120104&dsgk=0&dtdd=2"
}

options = Options()
options.add_experimental_option("mobileEmulation", mobile_emulation)
#options.add_argument('--headless')
options.binary_location = 'C:\\Program Files\\Google\\Chrome\\Application\\Chrome.exe'
chrome_service = ChromeService(executable_path='C:\\Users\\garden026\\WD\\114.0.5735.90\\chromedriver.exe')

gc = gspread.authorize(credentials)
wb = gc.open_by_key(SPREADSHEET_KEY)

worksheet = wb.worksheet("Config")
p4row = worksheet.cell(11,  23).value
p1row = worksheet.cell(11,  24).value
s20row = worksheet.cell(11,  28).value
s5row = worksheet.cell(11,  29).value

browser = webdriver.Chrome(service=chrome_service,options=options)
browser.command_executor._commands["send_command"] = ("POST", '/session/$sessionId/chromium/send_command')
params = {'cmd': 'Page.setDownloadBehavior', 'params': {'behavior': 'allow', 'downloadPath': 'site777'}}
command_result = browser.execute("send_command", params)

browser.get('https://m.site777.jp/f/A0100.do')
time.sleep(1)

browser.find_element("xpath",'/html/body/section[1]/div/div/p[3]/a').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/section/section[2]/div/div/p').click()
time.sleep(1)

browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[1]/input').send_keys(user_id)
browser.find_element("xpath",'/html/body/div[1]/div/section/div/dl/dd/form/div[2]/input').send_keys(user_pw)
time.sleep(1)

browser.execute_script('document.fmLogin.submit()')
time.sleep(1)

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(pstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

pmdc = browser.current_url.split("mdc=")[1]

browser.get('https://m.site777.jp/f/D0400.do?pmc=' + str(pmc))
time.sleep(1)

browser.find_element("xpath",'/html/body/div[2]/section/div[1]/dl/dd/form/input[1]').send_keys(sstart)
time.sleep(1)

browser.execute_script("S7Recaptcha.grv3FormSubmit(S7Recaptcha.GRV3_ACTION_HOMEPAGE,document.forms['formdaino'])")
time.sleep(1)

smdc = browser.current_url.split("mdc=")[1]

ck = browser.get_cookies()
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "PO_SP_ID":
        po_sp_id = ck[k]["value"]
        k = 30
    k = k + 1
k = 0
while k < 30:
    ccheck = ck[k]["name"]
    if ccheck == "JSESSIONID":
        jsessionid = ck[k]["value"]
        k = 30
    k = k + 1

cookie = {"PO_SP_ID" : po_sp_id,
         "JSESSIONID" : jsessionid}

browser.close()

p4table = []
p1table = []
s20table = []
s5table = []

# Pachinko

dn = pstart
mdc = pmdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D2600S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 100:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<hatariSu>")[m + cc].split("</hatariSu>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_p_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 341 or y > 343:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 342 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        game = round(dot_point_px * ( xx - 10) , 0)
                        samai = round(dot_point_p * point ,0)-300
                        if xx==10:
                            samai = 0
                            game = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass
                    
                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                print(li)
                if unit == "400":
                    p4table.append(li)
                if unit == "100":
                    p1table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb)).split("|")
                    print(li)
                    if unit == "400":
                        p4table.append(li)
                    if unit == "100":
                        p1table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

# Slot

dn = sstart
mdc = smdc

roop = 0
while roop < 1:
    try:
        url = "https://m.site777.jp/f/D3000S.do?pmc=" + str(pmc) + "&gc=2&dtdd=" + str(date) + "&dsgk=0&dn=" + str(dn) + "&mdc=" + str(mdc)
        response = requests.get(url=url, headers=headers, cookies=cookie)
        jsonData = response.text

        i = 1
        while i < 1000:
            try:
                temp = jsonData.split("<daiNo>")[i].split("</daiNo>")[0]
                if int(dn) == int(temp):
                    m = i
                    i = 1000
                i = i + 1
            except:
                i = 1000
                continue
            else:
                pass

        cc = 0
        while cc < 30:
            try:
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                unit = jsonData.split("<unit>")[m + cc].split("</unit>")[0]
                game = jsonData.split("<totalGameCount>")[m + cc].split("</totalGameCount>")[0]
                bb = jsonData.split("<bonusCount>")[m + cc].split("</bonusCount>")[0]
                rb = jsonData.split("<regularCount>")[m + cc].split("</regularCount>")[0]
                art = jsonData.split("<artCount>")[m + cc].split("</artCount>")[0]

                image = "https://m.site777.jp" + jsonData.split("<bigGraphUrl>")[m + cc].split("</bigGraphUrl>")[0].replace("amp;","")

                k = 0
                while(k < 10):
                    try:
                        filename =  'site777_2/dai_'+str(dn)+'.png'
                        download_img(image, filename)

                        time.sleep(wait)

                        Target_path = filename

                        template = cv2.imread(Template_s_path)
                        template_gray = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)

                        img_rgb = cv2.imread(Target_path)
                        img_gray = cv2.cvtColor(img_rgb, cv2.COLOR_BGR2GRAY)

                        res = cv2.matchTemplate(img_gray,template_gray,cv2.TM_CCOEFF_NORMED)

                        threshold = 0.6
                        loc = np.where( res >= threshold)

                        Mergin = 5
                        index = []
                        for iii in range(len(loc[1][:])):
                            count = 0
                            for j in range(len(loc[1][:])):
                                if abs(loc[1][:][iii] - loc[1][:][j]) < Mergin and abs(loc[0][:][iii] - loc[0][:][j]) < Mergin:
                                    if count > 0:
                                        index.append(j)
                                    count = count + 1

                        loc = np.delete(loc, index, axis=1)

                        w, h = template_gray.shape[::-1]

                        cv2.imwrite("site777_2/template_result.png",img_rgb)

                        img_bin = cv2.Canny(img_gray,128,200)
                        cv2.imwrite("site777_2/bin_result.png",img_bin)

                        count = 0

                        for x in range(0,234):
                            xx = 238 - x
                            a = 9
                            b = 442
                            if xx < 53:
                                a = 248
                                b = 435
                            for y in range(a,b):
                                if y < 332 or y > 334:
                                    if all(img_bin[y, xx]==[255, 255, 255]):
                                        point = 333 - y
                                        count = count + 1
                                        break
                            if count == 1:
                                break

                        samai = round(dot_point_s * point ,0)
                        if xx==10:
                            samai = 0

                        k = 10

                    except:
                        k = k + 1
                        continue

                    else:
                        pass

                temp = jsonData.split("<daiNo>")[m + cc + 1].split("</daiNo>")[0]
                li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                print(li)
                if unit == "2173":
                    s20table.append(li)
                if unit == "500":
                    s5table.append(li)
                cc = cc + 1

            except:
                if cc == 0:
                    li = (str(date2) + "|" + str(mdc) + "|" + str(dn) + "|" + str(game) + "|" + str(samai) + "|" + str(bb) + "|" + str(rb) + "|" + str(art)).split("|")
                    print(li)
                    if unit == "2173":
                        s20table.append(li)
                    if unit == "500":
                        s5table.append(li)
                    roop = 1
                dn = jsonData.split("<daiNo>")[m + cc].split("</daiNo>")[0]
                mdc = jsonData.split("<modelCode>")[m + cc].split("</modelCode>")[0]
                cc = 30
                continue
            else:
                pass
            
    except:
        continue
    else:
        pass

wb.values_append("'P4RAW'!F" + str(int(p4row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p4table})
wb.values_append("'P1RAW'!F" + str(int(p1row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": p1table})
wb.values_append("'S20RAW'!F" + str(int(s20row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s20table})
wb.values_append("'S5RAW'!F" + str(int(s5row)+1), {"valueInputOption": "USER_ENTERED"}, {"values": s5table})

['08/19', '025605', '1', '42821.0', '-1504.0', '43', '20']
['08/19', '025605', '2', '18974.0', '6320.0', '29', '10']
['08/19', '025605', '3', '34359.0', '2408.0', '40', '18']
['08/19', '025605', '4', '48718.0', '11436.0', '62', '20']
['08/19', '025605', '5', '30256.0', '5719.0', '43', '11']
['08/19', '025605', '6', '26667.0', '-2406.0', '22', '11']
['08/19', '025404', '7', '35897.0', '603.0', '42', '14']
['08/19', '025521', '8', '41795.0', '-7221.0', '32', '16']
['08/19', '025521', '9', '35128.0', '1506.0', '39', '16']
['08/19', '025521', '10', '37179.0', '11436.0', '55', '19']
['08/19', '025521', '11', '44103.0', '-3008.0', '39', '20']
['08/19', '025521', '12', '34615.0', '4214.0', '44', '18']
['08/19', '025765', '13', '58462.0', '-13240.0', '46', '21']
['08/19', '025765', '14', '38718.0', '-8726.0', '33', '13']
['08/19', '025765', '15', '45897.0', '603.0', '49', '22']
['08/19', '025765', '16', '53590.0', '-1203.0', '59', '24']
['08/19', '025765', '17', '37692.0', '1806.0', '48', '16'

['08/19', '025765', '140', '16923.0', '-4212.0', '14', '7']
['08/19', '025817', '141', '13590.0', '-2707.0', '29', '6']
['08/19', '025817', '142', '8974.0', '-5416.0', '5', '3']
['08/19', '025817', '143', '17949.0', '904.0', '48', '8']
['08/19', '025817', '144', '16923.0', '-7823.0', '16', '4']
['08/19', '025817', '145', '13846.0', '-2106.0', '32', '4']
['08/19', '025877', '146', '27179.0', '-1203.0', '28', '14']
['08/19', '025877', '147', '33333.0', '5719.0', '41', '15']
['08/19', '025877', '148', '21282.0', '6019.0', '32', '7']
['08/19', '025877', '149', '42051.0', '-3309.0', '43', '17']
['08/19', '025877', '150', '12564.0', '2709.0', '19', '7']
['08/19', '025892', '151', '21795.0', '-4814.0', '20', '7']
['08/19', '025892', '152', '7692.0', '-1504.0', '6', '3']
['08/19', '025892', '153', '11538.0', '-1203.0', '11', '4']
['08/19', '025892', '154', '6154.0', '1205.0', '9', '1']
['08/19', '025892', '155', '13846.0', '-7221.0', '7', '3']
['08/19', '025863', '156', '4615.0', '603.0', '8',

['08/19', '025873', '280', '11282.0', '-902.0', '8', '2']
['08/19', '025890', '721', '27692.0', '5418.0', '30', '8']
['08/19', '025890', '722', '1795.0', '-1203.0', '1', '1']
['08/19', '025890', '723', '3333.0', '7223.0', '11', '2']
['08/19', '025890', '724', '4359.0', '4214.0', '9', '1']
['08/19', '025890', '725', '8462.0', '-6619.0', '2', '1']
['08/19', '025869', '726', '19231.0', '2709.0', '19', '1']
['08/19', '025869', '727', '10256.0', '-8726.0', '1', '1']
['08/19', '025869', '728', '7949.0', '-7221.0', '0', '0']
['08/19', '025869', '729', '2051.0', '-1504.0', '1', '1']
['08/19', '025869', '730', '4615.0', '-4212.0', '0', '0']
['08/19', '025888', '731', '21282.0', '29492.0', '31', '7']
['08/19', '025888', '732', '6154.0', '5719.0', '7', '1']
['08/19', '025888', '733', '17692.0', '603.0', '10', '2']
['08/19', '025888', '734', '9231.0', '-1805.0', '4', '2']
['08/19', '025888', '735', '16667.0', '14746.0', '19', '3']
['08/19', '025888', '736', '10000.0', '27084.0', '23', '2']
['08/19

['08/19', '025856', '859', '18974.0', '5418.0', '15', '2']
['08/19', '025856', '860', '25641.0', '14445.0', '25', '5']
['08/19', '025856', '861', '43590.0', '1806.0', '27', '9']
['08/19', '025856', '862', '26667.0', '-18356.0', '4', '4']
['08/19', '025856', '863', '31538.0', '35209.0', '42', '5']
['08/19', '025856', '864', '24359.0', '-8425.0', '9', '6']
['08/19', '025856', '865', '35641.0', '11135.0', '29', '7']
['08/19', '025856', '866', '24103.0', '6922.0', '19', '6']
['08/19', '025856', '867', '34359.0', '-8425.0', '15', '3']
['08/19', '025856', '868', '28974.0', '-21666.0', '3', '2']
['08/19', '025856', '869', '20513.0', '11135.0', '20', '3']
['08/19', '025856', '870', '45385.0', '-11133.0', '20', '5']
['08/19', '025606', '871', '21538.0', '-4814.0', '8', '2']
['08/19', '025606', '872', '7949.0', '-6018.0', '1', '1']
['08/19', '025606', '873', '19231.0', '-14744.0', '3', '3']
['08/19', '025606', '874', '15641.0', '-12036.0', '2', '2']
['08/19', '025606', '875', '12821.0', '7825.0'

['08/19', '025824', '997', '31282.0', '6621.0', '44', '4']
['08/19', '025824', '998', '45128.0', '-19860.0', '22', '9']
['08/19', '025824', '999', '24359.0', '-16249.0', '4', '2']
['08/19', '025824', '1000', '34359.0', '-5416.0', '31', '7']
['08/19', '025579', '1001', '34103.0', '-6018.0', '12', '4']
['08/19', '025579', '1002', '26667.0', '15950.0', '24', '7']
['08/19', '025579', '1003', '23077.0', '603.0', '11', '2']
['08/19', '025579', '1004', '36154.0', '-15948.0', '9', '3']
['08/19', '025579', '1005', '24103.0', '-5416.0', '8', '3']
['08/19', '025579', '1006', '47436.0', '-3309.0', '21', '8']
['08/19', '025489', '1007', '44359.0', '5117.0', '21', '8']
['08/19', '025489', '1008', '43077.0', '-9629.0', '13', '7']
['08/19', '025489', '1009', '31026.0', '-11434.0', '7', '4']
['08/19', '025489', '1010', '48718.0', '-1504.0', '20', '4']
['08/19', '025489', '1011', '51538.0', '-15647.0', '13', '6']
['08/19', '025489', '1012', '46923.0', '-13842.0', '12', '5']
['08/19', '025436', '1013', '

['08/19', '120145', '369', '6679', '1019.0', '0', '10', '68']
['08/19', '120145', '370', '1769', '-278.0', '0', '2', '15']
['08/19', '120145', '371', '1683', '-1759.0', '0', '2', '6']
['08/19', '120145', '372', '5920', '2731.0', '0', '12', '74']
['08/19', '120145', '373', '897', '-370.0', '0', '1', '6']
['08/19', '120145', '374', '6160', '5231.0', '0', '8', '98']
['08/19', '120145', '375', '4495', '-880.0', '0', '10', '37']
['08/19', '120145', '376', '1838', '-1343.0', '0', '4', '9']
['08/19', '120145', '377', '4056', '278.0', '0', '4', '39']
['08/19', '120145', '378', '1693', '139.0', '0', '2', '17']
['08/19', '120145', '379', '2975', '-139.0', '0', '4', '26']
['08/19', '120145', '380', '6038', '-1111.0', '0', '10', '52']
['08/19', '120128', '381', '7227', '-3102.0', '0', '1', '18']
['08/19', '120128', '382', '7084', '7963.0', '0', '2', '33']
['08/19', '120128', '383', '5961', '12037.0', '0', '3', '34']
['08/19', '120128', '384', '5780', '4630.0', '0', '1', '24']
['08/19', '120128', '

['08/19', '120010', '501', '6779', '-833.0', '20', '26', '--']
['08/19', '120010', '502', '8254', '324.0', '31', '25', '--']
['08/19', '120010', '503', '6060', '-2824.0', '15', '10', '--']
['08/19', '120010', '504', '9119', '-231.0', '33', '22', '--']
['08/19', '120010', '505', '6811', '231.0', '27', '19', '--']
['08/19', '120010', '506', '7513', '648.0', '34', '19', '--']
['08/19', '120010', '507', '8486', '-1898.0', '29', '13', '--']
['08/19', '120010', '508', '5908', '-833.0', '21', '13', '--']
['08/19', '120010', '509', '7085', '-185.0', '27', '19', '--']
['08/19', '120010', '510', '8448', '2222.0', '37', '32', '--']
['08/19', '119996', '511', '7923', '1111.0', '32', '27', '--']
['08/19', '119996', '512', '7790', '509.0', '28', '28', '--']
['08/19', '119996', '513', '6872', '-1204.0', '22', '16', '--']
['08/19', '119996', '514', '5895', '185.0', '24', '16', '--']
['08/19', '119996', '515', '7837', '231.0', '29', '24', '--']
['08/19', '119996', '516', '7025', '139.0', '32', '10', '-

['08/19', '120088', '634', '4731', '2500.0', '24', '12', '15']
['08/19', '120139', '635', '2618', '1528.0', '0', '5', '25']
['08/19', '120139', '636', '1068', '-694.0', '0', '3', '4']
['08/19', '120053', '637', '2359', '-1528.0', '0', '5', '17']
['08/19', '120053', '638', '5383', '-1991.0', '0', '14', '50']
['08/19', '120131', '639', '371', '139.0', '0', '1', '10']
['08/19', '120131', '640', '2386', '324.0', '0', '7', '63']
['08/19', '120131', '641', '3255', '3102.0', '0', '18', '127']
['08/19', '120108', '642', '4785', '2500.0', '0', '0', '40']
['08/19', '120108', '643', '4915', '1759.0', '0', '0', '40']
['08/19', '120108', '644', '5333', '972.0', '0', '0', '36']
['08/19', '120108', '645', '2679', '-2037.0', '0', '0', '9']
['08/19', '120125', '646', '5397', '-1296.0', '18', '9', '--']
['08/19', '120125', '647', '3172', '-185.0', '10', '8', '--']
['08/19', '120125', '648', '2812', '-1111.0', '8', '2', '--']
['08/19', '120125', '649', '4581', '-463.0', '15', '9', '--']
['08/19', '120125

{'spreadsheetId': '1UI1bfGF8plOnDcJaIAPO7K__E22AeYNhHRAtIMrZzuo',
 'updates': {'spreadsheetId': '1UI1bfGF8plOnDcJaIAPO7K__E22AeYNhHRAtIMrZzuo',
  'updatedRange': 'S5RAW!F10196:M10235',
  'updatedRows': 40,
  'updatedColumns': 8,
  'updatedCells': 320}}